In [56]:
import tensorflow as tf
assert tf.__version__.startswith('2')

import os
import numpy as np
import matplotlib.pyplot as plt

In [57]:
tf.__version__

'2.2.0'

Sometimes the file is cannot be extracted. Run this in that case

In [58]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [59]:
!ls "/content/gdrive/Shared drives/Bangkit Data/Bany"

 12.ipynb
 cass
'Copy of MobileNetV2.ipynb'
'Copy of MobileNetV2_with_augmentation.ipynb'
'Copy of MobileNetV2_with_augmentation_maxpool_no_aug.ipynb'
 InceptionV3_with_augmentation_maxpool_no_aug_2.ipynb
 MobileNetV2_with_augmentation_maxpool_aug.ipynb
 MobileNetV2_with_augmentation_maxpool_no_aug.ipynb
 model2
 model3
 model_mobileNetV2
 ResNet152V2_no_aug.ipynb
 train
 train_augmentation
 VGG16_with_no_aug.ipynb
 VGG19_no_aug_3.ipynb


In [60]:
PATH = "/content/gdrive/Shared drives/Bangkit Data/Bany/train"

In [61]:
os.listdir(PATH)

['cmd', 'cbsd', 'cbb', 'healthy', 'cgm']

In [62]:
cbb_dir = os.path.join(PATH, 'cbb')
#cgm_augmentation_dir = os.path.join(PATH, 'cgm_augmentation')
cgm_dir = os.path.join(PATH, 'cgm')
#cbsd_augmentation_dir = os.path.join(PATH, 'cbsd_augmentation')
cbsd_dir = os.path.join(PATH, 'cbsd')
cmd_dir = os.path.join(PATH, 'cmd')
healthy_dir = os.path.join(PATH, 'cmd')

In [63]:
print(len(os.listdir(cbb_dir)))
print(len(os.listdir(cbsd_dir)))
print(len(os.listdir(cgm_dir)))
print(len(os.listdir(healthy_dir)))
print(len(os.listdir(cmd_dir)))

466
1443
773
2658
2658


In [64]:
import natsort
cbsd_list = natsort.natsorted(os.listdir(cbsd_dir))
len(cbsd_list)
cbsd_list[1]

'train-cbsd-1.jpg'

In [65]:
#import natsort
#dir_cbb = natsort.natsorted(os.listdir("/content/gdrive/Shared drives/Bangkit Data/Bany/train_augmentation/cbb"))
#dir_cbsd = natsort.natsorted(os.listdir("/content/gdrive/Shared drives/Bangkit Data/Bany/train_augmentation/cbsd"))
#dir_cgm = natsort.natsorted(os.listdir("/content/gdrive/Shared drives/Bangkit Data/Bany/train_augmentation/cgm"))
#dir_cmd = natsort.natsorted(os.listdir("/content/gdrive/Shared drives/Bangkit Data/Bany/train_augmentation/cmd"))
#dir_healthy = natsort.natsorted(os.listdir("/content/gdrive/Shared drives/Bangkit Data/Bany/train_augmentation/healthy"))
#print(len(dir_cbb))
#print(len(dir_cbsd))
#print(len(dir_cgm))
#print(len(dir_cmd))
#print(len(dir_healthy))


In [66]:
base_dir = PATH

In [67]:
IMAGE_SIZE = 224
BATCH_SIZE = 64

datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255, 
    validation_split=0.2)

train_generator = datagen.flow_from_directory(
    base_dir,
    target_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=BATCH_SIZE, 
    subset='training')

val_generator = datagen.flow_from_directory(
    base_dir,
    target_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=BATCH_SIZE, 
    subset='validation')

Found 4527 images belonging to 5 classes.
Found 1129 images belonging to 5 classes.


In [68]:
print (train_generator.class_indices)

{'cbb': 0, 'cbsd': 1, 'cgm': 2, 'cmd': 3, 'healthy': 4}


In [69]:
for image_batch, label_batch in train_generator:
  break
image_batch.shape, label_batch.shape

((64, 224, 224, 3), (64, 5))

In [ ]:
# Define the model

# Use no more than 2 Conv2D and 2 MaxPooling2D
model = tf.keras.models.Sequential([
    
    tf.keras.layers.Conv2D(32, (3,3), activation = 'relu', input_shape = (224, 224, 3)),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(32, (3,3), activation = 'relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(56, activation = 'relu'),
    tf.keras.layers.Dense(5, activation = 'softmax')])

# Compile Model. 
model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics=['accuracy'])

# Train the Model
history = model.fit(
train_generator, 
epochs=8,
validation_data=val_generator)



Epoch 1/8
71/71 [==============================] - 55s 771ms/step - loss: 1.9187 - accuracy: 0.2434 - val_loss: 1.5137 - val_accuracy: 0.4703
Epoch 2/8
71/71 [==============================] - 55s 769ms/step - loss: 1.4226 - accuracy: 0.4698 - val_loss: 1.3378 - val_accuracy: 0.4703
Epoch 3/8
71/71 [==============================] - 54s 760ms/step - loss: 1.3038 - accuracy: 0.4869 - val_loss: 1.2447 - val_accuracy: 0.5226
Epoch 4/8
71/71 [==============================] - 55s 774ms/step - loss: 1.1806 - accuracy: 0.5704 - val_loss: 1.1262 - val_accuracy: 0.5837
Epoch 5/8
19/71 [=======>......................] - ETA: 31s - loss: 1.0849 - accuracy: 0.5979

In [ ]:
model.summary()

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.legend(loc='upper left')
plt.ylabel('Accuracy')
plt.ylim([min(plt.ylim()),1])
plt.xlim(0,10)
plt.title('Training and Validation Accuracy')

plt.subplot(2, 1, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.legend(loc='upper left')
plt.ylabel('Cross Entropy')
plt.ylim([0,max(plt.ylim())])
plt.xlim(0,10)
plt.title('Training and Validation Loss')
plt.xlabel('epoch')
plt.show()

In [ ]:
import keras.backend as K

def precision(y_true, y_pred): 
    """Precision metric. Only computes a batch-wise average of precision.  
-    Computes the precision, a metric for multi-label classification of 
-    how many selected items are relevant. 
-    """ 
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1))) 
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1))) 
    precision = true_positives / (predicted_positives + K.epsilon()) 
    return precision 

def recall(y_true, y_pred): 
    """Recall metric. 
-    Only computes a batch-wise average of recall. 
-    Computes the recall, a metric for multi-label classification of 
-    how many relevant items are selected. 
-    """ 
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1))) 
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1))) 
    recall = true_positives / (possible_positives + K.epsilon()) 
    return recall 


def fbeta_score(y_true, y_pred, beta=1): 

    """Computes the F score.  
-    The F score is the weighted harmonic mean of precision and recall. 
-    Here it is only computed as a batch-wise average, not globally. 
-    This is useful for multi-label classification, where input samples can be 
-    classified as sets of labels. By only using accuracy (precision) a model 
-    would achieve a perfect score by simply assigning every class to every 
-    input. In order to avoid this, a metric should penalize incorrect class 
-    assignments as well (recall). The F-beta score (ranged from 0.0 to 1.0) 
-    computes this, as a weighted mean of the proportion of correct class 
-    assignments vs. the proportion of incorrect class assignments.  
-    With beta = 1, this is equivalent to a F-measure. With beta < 1, assigning 
-    correct classes becomes more important, and with beta > 1 the metric is 
-    instead weighted towards penalizing incorrect class assignments. 
-    """ 
    if beta < 0: 
        raise ValueError('The lowest choosable beta is zero (only precision).') 

    # If there are no true positives, fix the F score at 0 like sklearn. 
    if K.sum(K.round(K.clip(y_true, 0, 1))) == 0: 

        return 0 
    p = precision(y_true, y_pred) 
    r = recall(y_true, y_pred) 
    bb = beta ** 2 
    fbeta_score = (1 + bb) * (p * r) / (bb * p + r + K.epsilon()) 
    return fbeta_score 

def fmeasure(y_true, y_pred): 
    """Computes the f-measure, the harmonic mean of precision and recall. 
    Here it is only computed as a batch-wise average, not globally. 
    """ 
    return fbeta_score(y_true, y_pred, beta=1)

In [ ]:
!pip install tensorflowjs
import tensorflowjs as tfjs
from datetime import datetime
datestring = datetime.now().strftime("%Y%m%d%H%M%S")

     |████████████████████████████████| 61kB 1.8MB/s 
     |████████████████████████████████| 92kB 3.2MB/s 
     |████████████████████████████████| 144.4MB 101kB/s 
     |████████████████████████████████| 256kB 35.6MB/s 
     |████████████████████████████████| 921kB 35.6MB/s 
  Created wheel for PyInquirer: filename=PyInquirer-1.0.3-cp36-none-any.whl size=32851 sha256=c709c6eb25046446a4cad3c8704de10813e11973128976efbf9c4bf2b91039c6
  Stored in directory: /root/.cache/pip/wheels/52/6c/b1/3e4b0e8daf42a92883c7641c0ea8ffb62e0490ebed2faa55ad
Successfully built PyInquirer
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
  Found existing installation: tensorflow-hub 0.8.0
    Uninstalling tensorflow-hub-0.8.0:
      Successfully uninstalled tensorflow-hub-0.8.0
  Found existing installation: prompt-toolkit 1.0.18
    Uninstalling prompt-toolkit-1.0.18:
      Successfully uninstalled prompt-toolkit-1.0.18
  Found existing installation

/usr/local/lib/python3.6/dist-packages/tensorflowjs/converters/keras_h5_conversion.py:122: H5pyDeprecationWarning: The default file mode will change to 'r' (read-only) in h5py 3.0. To suppress this warning, pass the mode you need to h5py.File(), or set the global default h5.get_config().default_file_mode, or set the environment variable H5PY_DEFAULT_READONLY=1. Available modes are: 'r', 'r+', 'w', 'w-'/'x', 'a'. See the docs for details.
  return h5py.File(h5file)


In [ ]:
#tmp_path = "/tmp/keras_tfjs_{}".format(datestring)
#if os.path.isdir(tmp_path): shutil.rmtree(tmp_path)
#os.makedirs(tmp_path, exist_ok=True)
tfjs.converters.save_keras_model(model, "/content/gdrive/Shared drives/Bangkit Data/Valdi")

/usr/local/lib/python3.6/dist-packages/tensorflowjs/converters/keras_h5_conversion.py:122: H5pyDeprecationWarning: The default file mode will change to 'r' (read-only) in h5py 3.0. To suppress this warning, pass the mode you need to h5py.File(), or set the global default h5.get_config().default_file_mode, or set the environment variable H5PY_DEFAULT_READONLY=1. Available modes are: 'r', 'r+', 'w', 'w-'/'x', 'a'. See the docs for details.
  return h5py.File(h5file)
